# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

events_train = pd.read_parquet("events_train.par")
events_test = pd.read_parquet("events_test.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
users_train = events_train["user_id"].drop_duplicates().to_frame().reset_index(drop=True)
users_test = events_test["user_id"].drop_duplicates().to_frame().reset_index(drop=True)

common_users = users_train.merge(users_test,on="user_id",how="outer",indicator="indic")
common_users

,user_id,indic
0,1000000,left_only
1,1000001,left_only
2,1000002,left_only
3,1000003,both
4,1000004,left_only
...,...,...
430580,1430580,both
430581,1430581,left_only
430582,1430582,left_only
430583,1430583,left_only


In [5]:
cold_users = common_users[common_users["indic"] == "right_only"]["user_id"]

print(len(cold_users)) 

2365


In [6]:
cold_users

153       1000153
325       1000325
504       1000504
712       1000712
806       1000806
           ...   
429834    1429834
429875    1429875
430053    1430053
430311    1430311
430500    1430500
Name: user_id, Length: 2365, dtype: int64

In [7]:
events_train

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
0,1000000,5350,2016-03-18,2016-04-03,True,4,False,2016-03-01
1,1000000,6748,2016-04-16,2016-04-30,True,5,False,2016-04-01
2,1000000,17675462,2016-07-06,2016-07-15,True,5,False,2016-07-01
3,1000000,25494343,2016-06-10,2016-07-06,True,4,False,2016-06-01
4,1000000,17851885,2016-08-01,2016-08-09,True,4,False,2016-08-01
...,...,...,...,...,...,...,...,...
11751081,1430584,7896527,2016-05-18,2016-06-03,True,4,True,2016-05-01
11751082,1430584,29056083,2016-08-01,2016-08-03,True,3,True,2016-08-01
11751083,1430584,6614960,2015-11-02,2015-12-25,True,3,False,2015-11-01
11751084,1430584,3153910,2014-06-11,2014-07-01,True,5,False,2014-06-01


In [ ]:
# топ-100 наиболее популярных книг

top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity

In [ ]:
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]
item_popularity

In [ ]:
# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(by="popularity_weighted",ascending=False).reset_index(drop=True)
item_popularity

In [ ]:
# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity[item_popularity["avg_rating"]>=4].head(100).reset_index(drop=True)
top_k_pop_items

In [ ]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 10):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]])

In [ ]:
# Для какой доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items совпали по книгам?
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items, on="item_id", how="left")
cold_users_events_with_recs

In [ ]:
cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_items_no_avg_rating_idx

In [ ]:
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]]
cold_user_recs

In [ ]:
# Верный ответ — 0,8023 (без округления). Книги в top_k_pop_items настолько популярны, что большая часть «холодных» пользователей их читала! 
1-1912/9672

In [ ]:
# посчитаем метрики рекомендаций
# Посчитайте метрики rmse и mae для полученных рекомендаций.
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2))

# Верный ответ — 0,62. В среднем оценка рекомендации отклоняется от истинной на такую величину.
#  Для пятибалльной шкалы отклонение невысокое, но и метрика посчитана по популярным книгам с высокими оценками. 

In [ ]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

# «Холодных» пользователей без каких-либо релевантных рекомендаций — 57%,
#  то есть пересечение между оценёнными книгами и рекомендациями есть только у 43%, 
# по ним же и получено значение MAE-метрики. При этом среднее покрытие — 46%. 
# Это значит, что большая часть «холодных» пользователей не получила никаких релевантных рекомендаций, 
# а оставшаяся часть имеет пересечения только по 46% книг. 

# === Знакомство: первые персональные рекомендации

In [8]:
events.head()

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
0,1000000,5350,2016-03-18,2016-04-03,True,4,False,2016-03-01
1,1000000,6748,2016-04-16,2016-04-30,True,5,False,2016-04-01
2,1000000,17675462,2016-07-06,2016-07-15,True,5,False,2016-07-01
3,1000000,25494343,2016-06-10,2016-07-06,True,4,False,2016-06-01
4,1000000,17851885,2016-08-01,2016-08-09,True,4,False,2016-08-01


In [9]:
# Степень разреженности матрицы
print(len(events["rating"]))
print(len(items))
print(len(events["user_id"].drop_duplicates().to_frame().reset_index(drop=True)))

11751086
43312
430585


In [10]:
(43312*430585-11751086)/(43312*430585)

0.9993698979831817

# === Базовые подходы: коллаборативная фильтрация

In [11]:
# Реализация SVD-алгоритма

from surprise import Dataset, Reader
from surprise import SVD

In [6]:

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set

In [7]:
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [8]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [9]:
svd_predictions

[Prediction(uid=1000003, iid=25893709, r_ui=4, est=3.047716878985341, details={'was_impossible': False}),
 Prediction(uid=1000005, iid=34076952, r_ui=5, est=4.041735872183185, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=18812405, r_ui=3, est=3.999472296100204, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=7445, r_ui=4, est=4.41194063156509, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=29868610, r_ui=4, est=4.29199912216072, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=18774964, r_ui=4, est=4.642092319255585, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=24817626, r_ui=3, est=3.2471586563355213, details={'was_impossible': False}),
 Prediction(uid=1000007, iid=24453082, r_ui=3, est=3.7193525750864227, details={'was_impossible': False}),
 Prediction(uid=1000007, iid=28257707, r_ui=4, est=3.9574089125393153, details={'was_impossible': False}),
 Prediction(uid=1000007, iid=168642, r_ui=5, est=

In [10]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8259
MAE:  0.6458
0.8259326543219302 0.6458324731790988


In [11]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)
random_predictions

[Prediction(uid=1000003, iid=25893709, r_ui=4, est=5, details={'was_impossible': False}),
 Prediction(uid=1000005, iid=34076952, r_ui=5, est=4.3272637894387875, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=18812405, r_ui=3, est=4.878539300898058, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=7445, r_ui=4, est=5, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=29868610, r_ui=4, est=5, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=18774964, r_ui=4, est=3.014834713646897, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=24817626, r_ui=3, est=4.851241808254272, details={'was_impossible': False}),
 Prediction(uid=1000007, iid=24453082, r_ui=3, est=3.8017772802542478, details={'was_impossible': False}),
 Prediction(uid=1000007, iid=28257707, r_ui=4, est=3.8476438181957398, details={'was_impossible': False}),
 Prediction(uid=1000007, iid=168642, r_ui=5, est=4.3372123215973, details={'was_impossible': Fa

In [12]:
rmse = accuracy.rmse(random_predictions)
mae = accuracy.mae(random_predictions)
                     
print(rmse, mae) 

RMSE: 1.2608
MAE:  0.9999
1.2607921290603934 0.9998705396950534


In [18]:
events_test

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
50,1000003,25893709,2017-10-05,2017-10-17,True,4,False,2017-10-01
263,1000005,34076952,2017-10-09,2017-10-24,True,5,False,2017-10-01
285,1000006,18812405,2017-08-05,2017-08-19,True,3,False,2017-08-01
290,1000006,7445,2017-08-26,2017-08-30,True,4,False,2017-08-01
294,1000006,29868610,2017-08-30,2017-09-16,True,4,False,2017-08-01
...,...,...,...,...,...,...,...,...
11751007,1430579,27272506,2017-09-03,2017-10-07,True,3,True,2017-09-01
11751058,1430580,22021611,2017-10-05,2017-10-05,True,4,False,2017-10-01
11751059,1430580,15749186,2017-10-05,2017-10-18,True,4,False,2017-10-01
11751073,1430584,18692431,2017-08-02,2017-08-09,True,3,True,2017-08-01


In [24]:
# получить оценку для пользователя user_id и айтема item_id поможет метод predict
svd_model.predict("1000003","34076952")

Prediction(uid='1000003', iid='34076952', r_ui=None, est=3.9459914088879833, details={'was_impossible': False})

In [34]:
set(events_test[events_test["user_id"]==1000006]['item_id'].unique())

{7445, 18774964, 18812405, 24817626, 29868610}

In [44]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events_test["user_id"]==user_id]['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [svd_model.predict(user_id,item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"])

In [45]:
get_recommendations_svd(1296647, items, events_train, svd_model)

,item_id,score
0,24812,5.000000
1,323355,4.988212
2,30688013,4.975064
3,11737700,4.972143
4,6898978,4.960511


# Дополнительная проверка качества рекомендаций

In [46]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

user_id: 1309190


In [47]:
print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
29,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016-05-29,2016-07-04,5,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
30,Truman Capote,In Cold Blood,2015-06-03,2015-06-22,5,"{'Nonfiction': 7191, 'Classics': 4022, 'Crime-..."
31,T. Coraghessan Boyle,The Tortilla Curtain,2015-10-04,2015-10-19,3,"{'Fiction': 665, 'Contemporary': 76, 'Book Clu..."
32,Aaron Hartzler,What We Saw,2016-01-31,2016-02-21,5,"{'Young Adult': 480, 'Contemporary': 370, 'Mys..."
33,Jennifer L. Armentrout,"Opposition (Lux, #5)",2014-08-20,2014-08-23,5,"{'Young Adult': 1143, 'Fantasy-Paranormal': 95..."
34,"Cassandra Clare, Sarah Rees Brennan, Maureen J...",The Bane Chronicles,2015-01-08,2015-02-01,4,"{'Fantasy': 1972, 'Young Adult': 1142}"
35,Alexandra Bracken,"In The Afterlight (The Darkest Minds, #3)",2014-11-06,2014-11-08,5,"{'Young Adult': 989, 'Science Fiction-Dystopia..."
36,Jennifer L. Armentrout,"Beginnings: Obsidian & Onyx (Lux, #1-2)",2014-08-07,2014-08-08,4,"{'Young Adult': 140, 'Fantasy-Paranormal': 89,..."
37,Rick Riordan,"The Blood of Olympus (The Heroes of Olympus, #5)",2014-11-15,2015-02-15,4,"{'Fantasy': 4774, 'Fantasy-Mythology': 2114, '..."
38,"Krista Ritchie, Becca Ritchie","Addicted to You (Addicted, #1)",2016-03-27,2016-03-29,4,"{'New Adult': 1008, 'Romance': 783, 'Contempor..."


In [48]:
print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations)

Рекомендации


,item_id,score,author,title,genre_and_votes
0,1,5,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,"{'Fantasy': 46400, 'Young Adult': 15083, 'Fict..."
1,2,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Order of the Phoenix (Har...,"{'Fantasy': 46485, 'Young Adult': 15194, 'Fict..."
2,3,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,5,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."
4,6,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Goblet of Fire (Harry Pot...,"{'Fantasy': 48257, 'Young Adult': 15483, 'Fict..."


In [43]:
user_recommendations

,book_id,score
0,13,5
1,1111,5
2,13633498,5
3,33163378,5
4,8521879,5


# Коллаборативная фильтрация: ALS

In [12]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])


In [13]:
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

In [14]:
events_train[events_train["item_id"]==7445]

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,user_id_enc,item_id_enc
590,1000020,7445,2017-03-20,2017-03-30,True,3,False,2017-03-01,20,868
669,1000027,7445,2015-10-28,2015-12-01,True,5,False,2015-10-01,27,868
3229,1000085,7445,2012-08-04,2012-08-23,True,3,False,2012-08-01,85,868
9163,1000304,7445,2012-10-31,2012-11-03,True,1,False,2012-10-01,304,868
9453,1000316,7445,2012-11-21,2013-01-07,True,4,False,2012-11-01,316,868
...,...,...,...,...,...,...,...,...,...,...
11743759,1430319,7445,2015-05-23,2015-06-20,True,4,True,2015-05-01,430319,868
11745360,1430388,7445,2017-07-30,2017-08-13,True,5,False,2017-07-01,430388,868
11746281,1430416,7445,2012-03-12,2012-03-14,True,5,False,2012-03-01,430416,868
11746398,1430418,7445,2017-02-24,2017-06-20,True,5,False,2017-02-01,430418,868


In [15]:
items[items["item_id"]==7445]

,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,text_reviews_count,publisher,publication_year,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,item_id_enc
1795375,7445,Jeannette Walls,The Glass Castle,"A tender, moving tale of unconditional love in...","{'Nonfiction': 6451, 'Autobiography-Memoir': 5...",288,4.24,643450,40143,Scribner,2006,US,eng,Paperback,False,074324754X,9780743247542,"{'Academic': None, 'Academic-Academia': None, ...","Nonfiction 6451, Autobiography-Memoir 5734",868


In [16]:
# Матрица заняла бы 17 Гб даже для такого относительно небольшого набора данных
(43312*430585)/(1024*1024*1024)

17.36869804561138

In [17]:
# создаём sparse-матрицу формата CSR 
events_train["rating"] = events_train["rating"].astype(int)
events_train["user_id_enc"] = events_train["user_id_enc"].astype(int)
events_train["item_id_enc"] = events_train["item_id_enc"].astype(int)
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [18]:
# Размеры памяти, требуемой для хранения матрицы взаимодействий, уменьшились на два порядка
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3

0.26370687410235405

In [19]:
# создадим ALS-модель. Для примера возьмём количество латентных факторов для матриц $P, Q$, равным 50. 
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/anaconda3/lib/python3.12/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/home/conda/feedstock_root/build_artifacts/implicit_1724419594932/work/./implicit/gpu/utils.h:71)'
  warnings.warn(
/home/mle-user/anaconda3/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

In [20]:
# Чтобы получить рекомендации для пользователя с помощью модели ALS, используем такую функцию:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

In [23]:
get_recommendations_als(user_item_matrix_train, als_model, 1296647, user_encoder, item_encoder, include_seen=True, n=5)

,item_id_enc,score,item_id
0,27664,0.900479,9460487
1,38878,0.864531,22557272
2,1641,0.784560,13496
3,29910,0.691176,11870085
4,6942,0.679305,99561


In [26]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

In [27]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [28]:
als_recommendations.head()

,score,user_id,item_id
0,0.990940,1000000,3
1,0.896617,1000000,15881
2,0.864405,1000000,5
3,0.822256,1000000,6
4,0.774098,1000000,2


In [29]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet")

In [30]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
)

In [31]:
als_recommendations.head()

,user_id,item_id,score,rating_test
0,1000000,3,0.990940,NaN
1,1000000,15881,0.896617,NaN
2,1000000,5,0.864405,NaN
3,1000000,6,0.822256,NaN
4,1000000,2,0.774098,NaN


In [32]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

In [33]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
rating_test_idx


0           False
1           False
2           False
3           False
4           False
            ...  
43058495    False
43058496    False
43058497    False
43058498    False
43058499    False
Name: rating_test, Length: 43058500, dtype: bool

In [34]:
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))
ndcg_at_5_scores

/tmp/ipykernel_2152/732957696.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))


user_id
1000006    1.0
1000007    NaN
1000019    NaN
1000020    NaN
1000023    1.0
          ... 
1430558    NaN
1430569    1.0
1430573    NaN
1430578    1.0
1430584    NaN
Length: 48135, dtype: float64

In [35]:
print(ndcg_at_5_scores.mean())

0.9759562997103215


In [38]:
ndcg_at_5_scores

user_id
1000006    1.0
1000007    NaN
1000019    NaN
1000020    NaN
1000023    1.0
          ... 
1430558    NaN
1430569    1.0
1430573    NaN
1430578    1.0
1430584    NaN
Length: 48135, dtype: float64

In [39]:
17235/48135

0.3580554689934559

In [40]:
als_model.similar_items(3)

(array([    3,  1942,     4,     1,     0,     2,  8250, 26800, 23781,
        18446], dtype=int32),
 array([0.9999999 , 0.9975803 , 0.9962052 , 0.98827547, 0.982088  ,
        0.97494876, 0.97159123, 0.90773606, 0.90773606, 0.82279867],
       dtype=float32))

In [42]:
als_model.similar_users(20)

(array([    20, 205189, 123595, 241251, 236341, 329941,  48303, 303533,
        153470, 289638], dtype=int32),
 array([1.        , 0.8676769 , 0.8359941 , 0.8275909 , 0.8241378 ,
        0.8222106 , 0.8113861 , 0.81118315, 0.80413705, 0.8035676 ],
       dtype=float32))

# === Базовые подходы: контентные рекомендации

In [54]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)
items

TypeError: eval() arg 1 must be a string, bytes or code object

In [55]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v["genre_and_votes"]
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += 1
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

In [56]:
genres

,name,votes
genre_id,,
0,Womens Fiction-Chick Lit,1705
1,Fiction,22440
2,Politics,933
3,Humor,2101
4,Christian,1482
...,...,...
810,German History-Nazi Party,0
811,Favorites,0
812,History-Latin American History,0


In [57]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
1,Fiction,22440,0.106816
34,Romance,11166,0.053151
25,Fantasy,11108,0.052875
18,Young Adult,8614,0.041003
5,Nonfiction,6822,0.032473
52,Contemporary,5518,0.026266
16,Historical-Historical Fiction,5497,0.026166
20,Mystery,5110,0.024324
33,Fantasy-Paranormal,4413,0.021006


In [58]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

In [59]:
# Получим матрицу с весами по жанрам для каждой книги:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [65]:
# Аналогичным образом получим матрицу с весами по жанрам для какого-нибудь пользователя, например, для пользователя с идентификатором 1000010. 
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items)
user_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 149 stored elements and shape (22, 815)>

In [66]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [68]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
genre_id,,,
1,Fiction,22440,0.180513
38,Classics,4373,0.098675
25,Fantasy,11108,0.088185
5,Nonfiction,6822,0.044133
24,Science Fiction,3595,0.041981


In [69]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()
similarity_scores

array([0.55499991, 0.546343  , 0.5726901 , ..., 0.67143461, 0.03420267,
       0.33994595])

In [74]:
# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[::-1]
top_k_indices

array([ 4471,  1120, 14087, ..., 32709, 32708, 38761])

In [76]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
861044,J.K. Rowling,"Harry Potter and the Half-Blood Prince (Harry Potter, #6)","{'Fantasy': 46400, 'Young Adult': 15083, 'Fiction': 13083, 'Fantasy-Magic': 3815, 'Childrens': 2..."
2083381,"J.K. Rowling, Mary GrandPré","Harry Potter and the Order of the Phoenix (Harry Potter, #5)","{'Fantasy': 46485, 'Young Adult': 15194, 'Fiction': 13064}"
1584855,"J.K. Rowling, Mary GrandPré","Harry Potter and the Sorcerer's Stone (Harry Potter, #1)","{'Fantasy': 59818, 'Fiction': 17918, 'Young Adult': 17892, 'Fantasy-Magic': 4974, 'Childrens': 4..."
1028676,"J.K. Rowling, Mary GrandPré","Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)","{'Fantasy': 49784, 'Young Adult': 15393, 'Fiction': 14272, 'Fantasy-Magic': 4199, 'Childrens': 3..."
2251360,"J.K. Rowling, Mary GrandPré","Harry Potter and the Goblet of Fire (Harry Potter, #4)","{'Fantasy': 48257, 'Young Adult': 15483, 'Fiction': 13649}"
...,...,...,...
1229539,Lylah James,"The Mafia And His Angel: Part 2 (Tainted Hearts, #2)","{'Romance': 38, 'Dark': 37, 'Sociology-Abuse': 17}"
534649,David Anderson,The Remnant,{'Science Fiction Fantasy': 1}
1541052,Elaine Williams Crockett,Do Not Ask,"{'Fiction': 4, 'Mystery': 3, 'Thriller': 2}"
2251007,Logan Chance,Heartbreaker,"{'Romance': 20, 'Contemporary': 6}"


In [ ]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков